In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/minsearch.py

--2025-05-20 18:56:22--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.5’

minsearch.py.5      100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-05-20 18:56:22 (62.5 MB/s) - ‘minsearch.py.5’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc);

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
SELECT * WHERE course = 'data-engineering-zoomcamp';

SyntaxError: invalid syntax (3870143388.py, line 1)

In [8]:
q = 'I just started the course, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=1
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI()

In [15]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [16]:
response.choices[0].message.content

"I'm glad to hear you're interested in the course! Whether you can still enroll depends on the specific course's policies, deadlines, and availability. Here's what you can do:\n\n1. **Check the Course Website:** Look for any information on enrollment deadlines or late registration policies.\n   \n2. **Contact the Instructor or Administration:** Reach out to the course instructor or the institution offering the course to inquire about the possibility of late enrollment.\n\n3. **Look for Online Enrollment Options:** Some courses offer rolling admissions or self-paced modules that allow you to start at any time.\n\n4. **Review the Syllabus:** If you have access to the syllabus, check if any essential information or assignments have been missed and discuss with the instructor about catching up.\n\nIf the course has already started, it may still be possible to join if the policies allow for it. Good luck with your course!"

In [14]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [15]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [16]:
prompt = prompt_template.format(question=q, context=context).strip()

In [17]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: I just started the course, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [18]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, you can still enroll in the course even after the start date. You are eligible to submit the homework assignments. However, keep in mind that there will be deadlines for submitting the final projects, so it's important not to leave everything until the last minute."

In [19]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    
    return results

In [20]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [22]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [23]:
rag(query)

'To run Kafka for Java, navigate to the project directory and execute the following command in the terminal:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you\'re using Python and encountering the "Module \'kafka\' not found" issue for running `producer.py`, it\'s suggested to create a virtual environment, activate it, and install the required packages as specified in `requirements.txt`. Here\'s a quick guide to doing so:\n\n1. Create a virtual environment (you only need to do this once):\n   ```bash\n   python -m venv env\n   ```\n\n2. Activate the virtual environment (you\'ll need to do this each time you start a new terminal session):\n   - On MacOS/Linux:\n     ```bash\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```bash\n     env\\Scripts\\activate\n     ```\n\n3. Install dependencies:\n   ```bash\n   pip install -r ../requirements.txt\n   ```\n\nFinally, remember to deactivate the virtual 

In [24]:
from elasticsearch import Elasticsearch

In [25]:
es_client = Elasticsearch(
    'http://localhost:9200'
)

In [26]:
es_client.info()

ObjectApiResponse({'name': '32147bdb43dc', 'cluster_name': 'docker-cluster', 'cluster_uuid': '6yPjTR0fTPi3rBeRvh8RRg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [27]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [28]:
from tqdm.auto import tqdm

In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [30]:
 query = 'I just disovered the course. Can I still join it?'

In [37]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [39]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [40]:
rag(query)

'Yes, you can still join the course even after the start date. You are eligible to submit the homework without registration. However, please be aware of the deadlines for turning in the final projects to avoid last-minute submissions.'